In [1]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Complained object azqqwx does not exist in the database. Skipping.
Complained object ntmrkt does not exist in the database. Skipping.


In [3]:
import json
with open(r"D:\BEHAVIOR-1K\asset_pipeline\artifacts\pipeline\object_inventory.json", "r") as f:
    providers = {x.split("-")[1]: y for x, y in json.load(f)["providers"].items()}

In [4]:
import pathlib
all_meshes = []
for object_list_fn in pathlib.Path(r"D:\BEHAVIOR-1K\asset_pipeline").glob("cad/*/*/artifacts/object_list.json"):
    with open(object_list_fn, "r") as f:
        all_meshes.extend(json.load(f)["meshes"])

In [6]:
import sys
sys.path.append(r"D:\BEHAVIOR-1K\asset_pipeline")
from b1k_pipeline.utils import parse_name

# Automatically generate complaints for glassless windows
has_glass = set()
for mesh_name in all_meshes:
    if "Tglass" not in mesh_name:
        continue
    parsed_name = parse_name(mesh_name)
    assert parsed_name, mesh_name
    has_glass.add(parsed_name.group("model_id"))

In [10]:
for obj in Object.all_objects():
    complaints = []
    if obj.category.name in ("pot_plant", "floor_lamp", "table_lamp", "swivel_chair", "monitor", "standing_tv"):
        complaints.append({
            "object": str(obj),
            "message": "CENTEROFMASS: This object belongs to a category that frequently has stability issues. Please check the object's center of mass.",
            "complaint": "Auto-generated 2025",
            "processed": False,
        })
    
    if obj.category.name in ("fixed_window", "openable_window", "microwave", "oven"):
        # Does the window have a mesh that has a Tglass tag?
        if obj.name not in has_glass:
            complaints.append({
                "object": str(obj),
                "message": "GLASS: This window object does not have a glass-tagged mesh associated with it. Please check the object's meshes.",
                "complaint": "Auto-generated 2025",
                "processed": False,
            })

    if obj.category.synset.name == ("mirror.n.01"):
        complaints.append({
            "object": str(obj),
            "message": "MIRROR: This object is a mirror. Please ensure that the object is reflective.",
            "complaint": "Auto-generated 2025",
            "processed": False,
        })

    if any(meta_link.name == "slicer" for meta_link in obj.meta_links):
        complaints.append({
            "object": str(obj),
            "message": "SLICER: This object has a slicer meta link. Many objects of this type have issues with their positioning. Check it.",
            "complaint": "Auto-generated 2025",
            "processed": False,
        })

    if not complaints:
        continue

    obj_dir = pathlib.Path(r"D:\BEHAVIOR-1K\asset_pipeline\qa-2025\qa-autogen") / obj.category.name
    obj_dir.mkdir(parents=True, exist_ok=True)
    obj_file = obj_dir / f"{obj.name}.json"
    obj_data = {"orientation": [0., 0., 0., 1.], "scale": [1., 1., 1.], "complaints": complaints}
    obj_file.write_text(json.dumps(obj_data, indent=4))